# Environment Test Setup

## Imports

In [1]:
from kaggle_environments import evaluate, make, utils
import numpy as np

## Test Setup

In [ ]:
env = make("connectx", debug=True)
env.render()

In [ ]:
# 1. Create the environment
env = make("connectx", debug=True)

# 2. You must 'run' or 'reset' the environment for a board to exist
env.reset()

# 3. Explicitly print the human-readable render
print(env.render(mode="ansi"))

# Agent Definitions

## Random Agent

In [ ]:
def random_agent(observation, configuration):
    """
    A simple agent that plays a random, but valid, move.
    """
    import random
    # observation.board is a 1D list representing the grid.
    # 0 = empty, 1 = player 1, 2 = player 2.
    # The top row corresponds to indices 0 through (configuration.columns - 1).
    
    # Find all columns that are not completely full
    valid_moves = [
        col for col in range(configuration.columns) 
        if observation.board[col] == 0
    ]
    
    # Return a random choice among the valid columns
    return random.choice(valid_moves)

## Leftmost Agent

In [ ]:
def leftmost_agent(observation, configuration):
    """Always picks the leftmost valid column."""
    valid_moves = [col for col in range(configuration.columns) if observation.board[col] == 0]
    return valid_moves[0]

## Reactive Agent
1. Level 1: The One-Move Lookahead (Reactive)
The Goal: Build an agent that isn't blind. It should see a winning move and take it, and see an opponent’s winning move and block it.

Concepts: Simulating moves, identifying winning states, and basic heuristics.

Outcome: A "Smart-ish" agent that beats Random and Leftmost 100% of the time.

In [2]:
def reactive_agent(observation, configuration):
    """
    A Level 1 AI that identifies immediate wins and blocks immediate threats.
    All helper functions are defined inside or must be available in scope.
    
    Args:
        observation: Kaggle object (board, mark).
        configuration: Kaggle object (rows, columns, inarow).
        
    Returns:
        int: The column index to play.
    """
    import numpy as np
    import random

    # --- HELPER FUNCTIONS (Must be inside or accessible) ---
    def local_drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for r in range(config.rows - 1, -1, -1):
            if next_grid[r][col] == 0:
                next_grid[r][col] = piece
                return next_grid
        return next_grid

    def local_is_win(grid, piece, config):
        # Horizontal
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                window = list(grid[r, c:c+config.inarow])
                if window.count(piece) == config.inarow: return True
        # Vertical
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow - 1)):
                window = list(grid[r:r+config.inarow, c])
                if window.count(piece) == config.inarow: return True
        # Positive Diagonal
        for r in range(config.rows - (config.inarow - 1)):
            for c in range(config.columns - (config.inarow - 1)):
                window = [grid[r+i, c+i] for i in range(config.inarow)]
                if window.count(piece) == config.inarow: return True
        # Negative Diagonal
        for r in range(config.inarow - 1, config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                window = [grid[r-i, c+i] for i in range(config.inarow)]
                if window.count(piece) == config.inarow: return True
        return False

    # --- AGENT LOGIC ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark
    enemy = 1 if me == 2 else 2
    
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]

    # 1. Check for Win
    for col in valid_moves:
        potential_grid = local_drop_piece(grid, col, me, configuration)
        if local_is_win(potential_grid, me, configuration):
            return int(col) # Force standard Python int

    # 2. Check for Block
    for col in valid_moves:
        potential_grid = local_drop_piece(grid, col, enemy, configuration)
        if local_is_win(potential_grid, enemy, configuration):
            return int(col) # Force standard Python int

    # 3. Random fallback
    return int(random.choice(valid_moves))

### Functions for Testing

In [ ]:
# For unit test purpose
def drop_piece(grid, col, piece, config):
    """
    Simulates dropping a piece into a specific column of the board.

    Args:
        grid (np.ndarray): A 2D numpy array representing the current board state.
        col (int): The index of the column (0 to config.columns-1) where the piece is dropped.
        piece (int): The player's mark (1 or 2).
        config (struct): The game configuration containing board dimensions (rows, columns).

    Returns:
        np.ndarray: A new 2D numpy array representing the board AFTER the move.
                   Returns the original grid if the move is invalid (column full).
    """
    # Create a deep copy so we don't modify the original 'real' board
    next_grid = grid.copy()
    
    # We loop from the bottom row up to the top row
    # range(start, stop, step) -> (bottom_index, -1, go_up_by_one)
    for row in range(config.rows - 1, -1, -1):
        if next_grid[row][col] == 0:
            next_grid[row][col] = piece
            return next_grid
            
    # If we get here, the column was full
    return next_grid

In [ ]:
# for unit test purpose
def is_win(grid, piece, config):
    """
    Scans the board to determine if the specified player has won.

    Args:
        grid (np.ndarray): The 2D numpy array representing the board.
        piece (int): The player mark to check for (1 or 2).
        config (struct): Configuration containing 'rows', 'columns', and 'inarow'.

    Returns:
        bool: True if the player has 'config.inarow' in a line, False otherwise.
    """
    # 1. Check Horizontal
    for r in range(config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = list(grid[r, c:c+config.inarow])
            if window.count(piece) == config.inarow:
                return True

    # 2. Check Vertical
    for c in range(config.columns):
        for r in range(config.rows - (config.inarow - 1)):
            window = list(grid[r:r+config.inarow, c])
            if window.count(piece) == config.inarow:
                return True

    # 3. Check Positive Diagonal (/)
    for r in range(config.rows - (config.inarow - 1)):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r+i, c+i] for i in range(config.inarow)]
            if window.count(piece) == config.inarow:
                return True

    # 4. Check Negative Diagonal (\)
    for r in range(config.inarow - 1, config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r-i, c+i] for i in range(config.inarow)]
            if window.count(piece) == config.inarow:
                return True

    return False

### Unit Tests

In [ ]:
# Test drop_piece()
class MockConfig:
    def __init__(self, rows, columns):
        self.rows = rows
        self.columns = columns

def test_drop_piece_advanced():
    config_2x2 = MockConfig(rows=2, columns=2)
    board = np.zeros((2, 2), dtype=int)
    print("Step 0: Empty 2x2 Board\n", board)
    
    # 1) Drop piece 1 in column 0
    board = drop_piece(board, col=0, piece=1, config=config_2x2)
    print("\nStep 1: P1 in Col 0\n", board)
    assert board[1][0] == 1
    
    # 2) Drop piece 2 in column 1
    board = drop_piece(board, col=1, piece=2, config=config_2x2)
    print("\nStep 2: P2 in Col 1\n", board)
    assert board[1][1] == 2
    
    # 3) Drop piece 1 in column 0
    board = drop_piece(board, col=0, piece=1, config=config_2x2)
    print("\nStep 3: P1 in Col 0 (should stack)\n", board)
    assert board[0][0] == 1
    
    # 4) Drop piece 2 in column 0 (Wait, this column is already full!)
    # Since it's a 2x2 board, Column 0 now has two pieces. 
    # Let's see how our function handles a 3rd piece in a 2-high column.
    board_after_full = drop_piece(board, col=0, piece=2, config=config_2x2)
    print("\nStep 4: P2 in FULL Col 0 (should not change board)\n", board_after_full)
    
    # Verification
    assert np.array_equal(board, board_after_full), "Error: Board changed even though column was full!"
    print("\n✅ All tests passed! The simulator correctly handles stacking and full columns.")

test_drop_piece_advanced()

In [ ]:
# Test is_win()
class MockConfig:
    def __init__(self, rows, columns, inarow):
        self.rows = rows
        self.columns = columns
        self.inarow = inarow

def test_is_win():
    # Setup: 3x3 board where you only need 2-in-a-row to win
    config_3x3 = MockConfig(rows=3, columns=3, inarow=2)
    
    # Test 1: Horizontal Win
    grid_h = np.array([
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 0]  # Two 1s in the bottom row
    ])
    assert is_win(grid_h, 1, config_3x3) == True
    assert is_win(grid_h, 2, config_3x3) == False # Player 2 hasn't won
    print("Test 1: Horizontal Passed")

    # Test 2: Vertical Win
    grid_v = np.array([
        [2, 0, 0],
        [2, 0, 0],
        [1, 0, 0]
    ])
    assert is_win(grid_v, 2, config_3x3) == True
    print("Test 2: Vertical Passed")

    # Test 3: Positive Diagonal (/)
    grid_pd = np.array([
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 0]
    ])
    assert is_win(grid_pd, 1, config_3x3) == True
    print("Test 3: Positive Diagonal Passed")

    # Test 4: Negative Diagonal (\)
    grid_nd = np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 0]
    ])
    assert is_win(grid_nd, 2, config_3x3) == True
    print("Test 4: Negative Diagonal Passed")

    # Test 5: No Win
    grid_none = np.array([
        [1, 2, 1],
        [2, 1, 2],
        [1, 2, 1]
    ])
    # Even though it's full, nobody has 2-in-a-row? 
    # Wait, in a 3x3 with 2-in-a-row, almost any move wins.
    # Let's use an empty board.
    grid_empty = np.zeros((3,3))
    assert is_win(grid_empty, 1, config_3x3) == False
    print("Test 5: Empty Board Passed")

    print("\n✅ All Vision Tests Passed! The agent can now see.")

test_is_win()

In [ ]:
class MockObject:
    """A simple class to turn a dictionary into an object with dot-notation access."""
    def __init__(self, **entries):
        self.__dict__.update(entries)

def test_reactive_agent_blocking():
    """
    Scenario: The enemy (Player 2) has 3-in-a-row horizontally on the bottom row.
    The reactive_agent (Player 1) MUST block the 4th spot.
    
    Args:
        None
        
    Returns:
        None: Prints success or raises an AssertionError.
    """
    # 1. Setup Configuration using our MockObject
    config = MockObject(rows=6, columns=7, inarow=4)
    
    # 2. Construct the Board State
    # Bottom row (index 5): [2, 2, 2, 0, 0, 0, 0]
    grid = np.zeros((6, 7), dtype=int)
    grid[5][0] = 2
    grid[5][1] = 2
    grid[5][2] = 2
    
    # 3. Create the Observation using our MockObject
    # This ensures observation.board and observation.mark work correctly!
    obs = MockObject(board=list(grid.flatten()), mark=1)
    
    # 4. Execute Agent Logic
    chosen_col = reactive_agent(obs, config)
    
    # 5. Verification
    print(f"Enemy has 3-in-a-row in cols 0,1,2. Agent chose: {chosen_col}")
    assert chosen_col == 3, f"Fail: Agent should have blocked col 3, but chose {chosen_col}"
    print("✅ Blocking Scenario Passed!")

def test_reactive_agent_winning():
    """
    Scenario: The agent (Player 1) has 3 pieces stacked vertically in Column 4.
    The agent MUST recognize that playing in Column 4 results in an immediate win.
    
    Args:
        None (Uses the MockObject class and existing reactive_agent).
        
    Returns:
        None: Prints success or raises an AssertionError.
    """
    # 1. Setup Configuration using our MockObject
    config = MockObject(rows=6, columns=7, inarow=4)
    
    # 2. Construct the Board State
    # Three '1's in column 4, starting from the bottom.
    grid = np.zeros((6, 7), dtype=int)
    grid[5][4] = 1 
    grid[4][4] = 1 
    grid[3][4] = 1 
    
    # 3. Create the Observation using our MockObject
    obs = MockObject(board=list(grid.flatten()), mark=1)
    
    # 4. Execute Agent Logic
    chosen_col = reactive_agent(obs, config)
    
    # 5. Verification
    print(f"Agent (P1) sees 3-in-a-row in Col 4. Agent chose: {chosen_col}")
    assert chosen_col == 4, f"Fail: Agent should have won in Col 4, but chose {chosen_col}"
    print("✅ Winning Scenario Passed!")

# Run the test
test_reactive_agent_blocking()
test_reactive_agent_winning()

## Strategic Agent (MinMax)

Level 2: The Minimax Tree (Strategic)The Goal: Teach the agent to think ahead $N$ moves. It will assume the opponent is also playing optimally and choose the path that leads to the best guaranteed outcome.
Concepts: Recursion, the "Min" and "Max" players, and Depth-Limited Search.
Outcome: An agent that can compete with the built-in negamax.

### Agent Code

In [2]:
def minimax_agent(observation, configuration):
    """
    A Level 2 Strategic AI using Depth-Limited Minimax.
    Contains all necessary logic to be fully self-contained for Kaggle.
    """
    import numpy as np
    import random

    # --- SETTINGS ---
    DEPTH = 3 # Lookahead depth
    
    # --- HELPER: DROP PIECE ---
    def local_drop_piece(grid, col, piece, config):
        next_grid = grid.copy()
        for r in range(config.rows - 1, -1, -1):
            if next_grid[r][col] == 0:
                next_grid[r][col] = piece
                return next_grid
        return next_grid

    # --- HELPER: IS WIN ---
    def local_is_win(grid, piece, config):
        # Horizontal
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r, c+i] == piece for i in range(config.inarow)): return True
        # Vertical
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow - 1)):
                if all(grid[r+i, c] == piece for i in range(config.inarow)): return True
        # Diagonals
        for r in range(config.rows - (config.inarow - 1)):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r+i, c+i] == piece for i in range(config.inarow)): return True
        for r in range(config.inarow - 1, config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r-i, c+i] == piece for i in range(config.inarow)): return True
        return False

    # --- HELPER: HEURISTIC SCORER ---
    def local_get_score(grid, piece, config):
        score = 0
        enemy = 1 if piece == 2 else 2
        
        def evaluate_window(window, p, e):
            w_score = 0
            if window.count(p) == 4: w_score += 1000000
            elif window.count(p) == 3 and window.count(0) == 1: w_score += 100
            elif window.count(p) == 2 and window.count(0) == 2: w_score += 10
            if window.count(e) == 3 and window.count(0) == 1: w_score -= 10000
            if window.count(e) == 4: w_score -= 1000000
            return w_score

        # Scan Horizontal, Vertical, Diagonals (Simplified for brevity)
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window(list(grid[r, c:c+config.inarow]), piece, enemy)
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow-1)):
                score += evaluate_window(list(grid[r:r+config.inarow, c]), piece, enemy)
        for r in range(config.rows - (config.inarow-1)):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r+i, c+i] for i in range(config.inarow)], piece, enemy)
        for r in range(config.inarow-1, config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r-i, c+i] for i in range(config.inarow)], piece, enemy)
        return score

    # --- CORE: RECURSIVE MINIMAX ---
    def local_minimax(grid, depth, is_maximizing, piece, config):
        enemy = 1 if piece == 2 else 2
        
        if local_is_win(grid, piece, config): return 1000000
        if local_is_win(grid, enemy, config): return -1000000
        if depth == 0: return local_get_score(grid, piece, config)
        
        valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        if not valid_moves: return 0

        if is_maximizing:
            best_score = -float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, piece, config)
                score = local_minimax(temp_grid, depth - 1, False, piece, config)
                best_score = max(score, best_score)
            return best_score
        else:
            best_score = float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, enemy, config)
                score = local_minimax(temp_grid, depth - 1, True, piece, config)
                best_score = min(score, best_score)
            return best_score

    # --- AGENT EXECUTION ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    
    best_score = -float('inf')
    best_move = random.choice(valid_moves)
    
    for col in valid_moves:
        temp_grid = local_drop_piece(grid, col, me, configuration)
        score = local_minimax(temp_grid, DEPTH-1, False, me, configuration)
        if score > best_score:
            best_score = score
            best_move = col
            
    return int(best_move)

### Unit Tests

In [ ]:
class MockObject:
    """A simple class to turn a dictionary into an object with dot-notation access."""
    def __init__(self, **entries):
        self.__dict__.update(entries)

def get_score(grid, piece, config):
    """
    Evaluates the 'goodness' of a board state for a specific player.
    Note: 'Double counting' overlapping windows is a feature that rewards 
    centrality and connectivity.
    """
    enemy = 1 if piece == 2 else 2
    score = 0
    
    def evaluate_window(window, piece, config):
        window_score = 0
        enemy = 1 if piece == 2 else 2
        
        # --- MY SCORES ---
        if window.count(piece) == 4:
            window_score += 1000000   # Victory
        elif window.count(piece) == 3 and window.count(0) == 1:
            window_score += 100       # Strong potential
        elif window.count(piece) == 2 and window.count(0) == 2:
            window_score += 10        # Building blocks
            
        # --- ENEMY SCORES ---
        if window.count(enemy) == 4:
            window_score -= 1000000   # Immediate Loss (Hard Floor)
        elif window.count(enemy) == 3 and window.count(0) == 1:
            window_score -= 10000     # Critical Threat (Must Block)
            
        return window_score

    # 1. Horizontal Score
    for r in range(config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = list(grid[r, c:c+config.inarow])
            score += evaluate_window(window, piece, config)

    # 2. Vertical Score
    for c in range(config.columns):
        for r in range(config.rows - (config.inarow - 1)):
            window = list(grid[r:r+config.inarow, c])
            score += evaluate_window(window, piece, config)

    # 3. Positive Diagonal (/) Score
    for r in range(config.rows - (config.inarow - 1)):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r+i, c+i] for i in range(config.inarow)]
            score += evaluate_window(window, piece, config)

    # 4. Negative Diagonal (\) Score
    for r in range(config.inarow - 1, config.rows):
        for c in range(config.columns - (config.inarow - 1)):
            window = [grid[r-i, c+i] for i in range(config.inarow)]
            score += evaluate_window(window, piece, config)

    return score

def test_heuristic_scoring_full():
    """
    Checks the scoring hierarchy: 
    Winning > Blocking Enemy > Having 3-in-a-row > Having 2-in-a-row.
    """
    config = MockObject(rows=6, columns=7, inarow=4)
    
    # 1. My 2-in-a-row
    grid_2 = np.zeros((6, 7), dtype=int)
    grid_2[5, 0:2] = 1
    score_2 = get_score(grid_2, 1, config)
    
    # 2. My 3-in-a-row
    grid_3 = np.zeros((6, 7), dtype=int)
    grid_3[5, 0:3] = 1
    score_3 = get_score(grid_3, 1, config)
    
    # 3. Enemy's 3-in-a-row (Danger!)
    grid_enemy_3 = np.zeros((6, 7), dtype=int)
    grid_enemy_3[5, 0:3] = 2 
    score_enemy_3 = get_score(grid_enemy_3, 1, config)
    
    # 4. My 4-in-a-row (Victory)
    grid_4 = np.zeros((6, 7), dtype=int)
    grid_4[5, 0:4] = 1
    score_4 = get_score(grid_4, 1, config)
    
    # 5. Enemy's 4-in-a-row (Loss)
    grid_enemy_4 = np.zeros((6, 7), dtype=int)
    grid_enemy_4[5, 0:4] = 2
    score_enemy_4 = get_score(grid_enemy_4, 1, config)

    print(f"--- Heuristic Scoring Results ---")
    print(f"My 4-in-a-row:     {score_4}")
    print(f"My 3-in-a-row:     {score_3}")
    print(f"My 2-in-a-row:     {score_2}")
    print(f"Enemy 3-in-a-row:  {score_enemy_3}")
    print(f"Enemy 4-in-a-row:  {score_enemy_4}")

    # Hierarchy Assertions
    assert score_4 > score_3 > score_2
    assert score_enemy_3 < 0
    assert score_enemy_4 < score_enemy_3
    print("\n✅ All Heuristic Hierarchy Tests Passed!")

# Run the test
test_heuristic_scoring_full()

## Optimized Agent (Alpha Beta Pruning)

3. Level 3: Alpha-Beta Pruning (Optimized)
The Goal: Make the search faster. Right now, Minimax looks at every single branch. Pruning allows us to ignore branches that are obviously worse, letting us look 6–8 moves ahead instead of just 3.

Concepts: Branch cutting and search efficiency.

Outcome: A competitive agent ready for the Kaggle leaderboard.

### Alpha Beta (Depth=4)

In [3]:
def alphabeta_d4_agent(observation, configuration):
    """
    A Level 3 Strategic AI using Depth-Limited Minimax with Alpha-Beta Pruning.
    
    This agent evaluates the game tree to a specified depth, using Alpha-Beta
    pruning to discard branches that cannot improve the final decision. It 
    prioritizes central columns to maximize pruning efficiency.
    
    Args:
        observation: Kaggle object containing 'board' (1D list) and 'mark' (1 or 2).
        configuration: Kaggle object containing 'rows', 'columns', and 'inarow'.
        
    Returns:
        int: The selected column index (0 to configuration.columns-1).
    """
    import numpy as np
    import random

    # --- SETTINGS ---
    # Alpha-beta typically allows for Depth 4-5 within the 2s time limit.
    DEPTH = 4 

    # --- HELPER: DROP PIECE ---
    def local_drop_piece(grid, col, piece, config):
        """
        Simulates the gravity-fed 'drop' of a piece into the board.
        
        Args:
            grid (np.ndarray): The current 2D board state.
            col (int): The column index to drop the piece into.
            piece (int): The player's mark (1 or 2).
            config: Game configuration for board dimensions.
            
        Returns:
            np.ndarray: A copy of the grid with the new piece placed.
        """
        next_grid = grid.copy()
        for r in range(config.rows - 1, -1, -1):
            if next_grid[r][col] == 0:
                next_grid[r][col] = piece
                return next_grid
        return next_grid

    # --- HELPER: IS WIN ---
    def local_is_win(grid, piece, config):
        """
        Determines if the specified piece has achieved the required N-in-a-row.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The player mark to check (1 or 2).
            config: Game configuration for 'inarow' requirement.
            
        Returns:
            bool: True if a winning connection is found, False otherwise.
        """
        # Horizontal check
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r, c+i] == piece for i in range(config.inarow)): return True
        # Vertical check
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow - 1)):
                if all(grid[r+i, c] == piece for i in range(config.inarow)): return True
        # Positive Diagonal check
        for r in range(config.rows - (config.inarow - 1)):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r+i, c+i] == piece for i in range(config.inarow)): return True
        # Negative Diagonal check
        for r in range(config.inarow - 1, config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r-i, c+i] == piece for i in range(config.inarow)): return True
        return False

    # --- HELPER: HEURISTIC SCORER ---
    def local_get_score(grid, piece, config):
        """
        Evaluates the strategic value of a non-terminal board state.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The agent's mark.
            config: Game configuration for window scanning.
            
        Returns:
            float: A weighted score representing board 'goodness'.
        """
        score = 0
        enemy = 1 if piece == 2 else 2
        
        def evaluate_window(window, p, e):
            w_score = 0
            # Prioritize wins
            if window.count(p) == 4: w_score += 1000000
            # Reward setups
            elif window.count(p) == 3 and window.count(0) == 1: w_score += 100
            elif window.count(p) == 2 and window.count(0) == 2: w_score += 10
            # Penalize enemy threats heavily
            if window.count(e) == 3 and window.count(0) == 1: w_score -= 10000
            if window.count(e) == 4: w_score -= 1000000
            return w_score

        # Scan Horizontal
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window(list(grid[r, c:c+config.inarow]), piece, enemy)
        # Scan Vertical
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow-1)):
                score += evaluate_window(list(grid[r:r+config.inarow, c]), piece, enemy)
        # Scan Diagonals
        for r in range(config.rows - (config.inarow-1)):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r+i, c+i] for i in range(config.inarow)], piece, enemy)
        for r in range(config.inarow-1, config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r-i, c+i] for i in range(config.inarow)], piece, enemy)
        return score

    # --- CORE: ALPHA-BETA RECURSION ---
    def alphabeta(grid, depth, alpha, beta, is_maximizing, piece, config):
        """
        Executes the recursive search with pruning logic.
        
        Args:
            grid (np.ndarray): Current hypothetical board.
            depth (int): Current depth in search tree.
            alpha (float): Best score found for Maximizer.
            beta (float): Best score found for Minimizer.
            is_maximizing (bool): Whose turn it is in the simulation.
            piece (int): The agent's mark.
            config: Game configuration.
            
        Returns:
            float: The heuristic or terminal value of the branch.
        """
        enemy = 1 if piece == 2 else 2
        
        # Base Cases
        if local_is_win(grid, piece, config): return 1000000
        if local_is_win(grid, enemy, config): return -1000000
        if depth == 0: return local_get_score(grid, piece, config)
        
        valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        if not valid_moves: return 0

        if is_maximizing:
            value = -float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, piece, config)
                value = max(value, alphabeta(temp_grid, depth - 1, alpha, beta, False, piece, config))
                alpha = max(alpha, value)
                if alpha >= beta: break 
            return value
        else:
            value = float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, enemy, config)
                value = min(value, alphabeta(temp_grid, depth - 1, alpha, beta, True, piece, config))
                beta = min(beta, value)
                if alpha >= beta: break
            return value

    # --- AGENT EXECUTION ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark
    
    # 1. Get and sort moves (Center-out heuristic)
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    valid_moves = sorted(valid_moves, key=lambda x: abs(x - (configuration.columns // 2)))

    best_score = -float('inf')
    best_move = valid_moves[0]
    
    # 2. Root-Level Pruning: Initialize Alpha and Beta at the top
    alpha = -float('inf')
    beta = float('inf')
    
    for col in valid_moves:
        temp_grid = local_drop_piece(grid, col, me, configuration)
        # Check the score of this move
        score = alphabeta(temp_grid, DEPTH-1, alpha, beta, False, me, configuration)
        
        if score > best_score:
            best_score = score
            best_move = col
        
        # Update alpha for the next column check
        alpha = max(alpha, best_score)
            
    return int(best_move)

### Alpha Beta (Depth=5)

In [4]:
def alphabeta_d5_agent(observation, configuration):
    """
    A Level 3 Strategic AI using Depth-Limited Minimax with Alpha-Beta Pruning.
    
    This agent evaluates the game tree to a specified depth, using Alpha-Beta
    pruning to discard branches that cannot improve the final decision. It 
    prioritizes central columns to maximize pruning efficiency.
    
    Args:
        observation: Kaggle object containing 'board' (1D list) and 'mark' (1 or 2).
        configuration: Kaggle object containing 'rows', 'columns', and 'inarow'.
        
    Returns:
        int: The selected column index (0 to configuration.columns-1).
    """
    import numpy as np
    import random

    # --- SETTINGS ---
    # Alpha-beta typically allows for Depth 4-5 within the 2s time limit.
    DEPTH = 5 

    # --- HELPER: DROP PIECE ---
    def local_drop_piece(grid, col, piece, config):
        """
        Simulates the gravity-fed 'drop' of a piece into the board.
        
        Args:
            grid (np.ndarray): The current 2D board state.
            col (int): The column index to drop the piece into.
            piece (int): The player's mark (1 or 2).
            config: Game configuration for board dimensions.
            
        Returns:
            np.ndarray: A copy of the grid with the new piece placed.
        """
        next_grid = grid.copy()
        for r in range(config.rows - 1, -1, -1):
            if next_grid[r][col] == 0:
                next_grid[r][col] = piece
                return next_grid
        return next_grid

    # --- HELPER: IS WIN ---
    def local_is_win(grid, piece, config):
        """
        Determines if the specified piece has achieved the required N-in-a-row.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The player mark to check (1 or 2).
            config: Game configuration for 'inarow' requirement.
            
        Returns:
            bool: True if a winning connection is found, False otherwise.
        """
        # Horizontal check
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r, c+i] == piece for i in range(config.inarow)): return True
        # Vertical check
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow - 1)):
                if all(grid[r+i, c] == piece for i in range(config.inarow)): return True
        # Positive Diagonal check
        for r in range(config.rows - (config.inarow - 1)):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r+i, c+i] == piece for i in range(config.inarow)): return True
        # Negative Diagonal check
        for r in range(config.inarow - 1, config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r-i, c+i] == piece for i in range(config.inarow)): return True
        return False

    # --- HELPER: HEURISTIC SCORER ---
    def local_get_score(grid, piece, config):
        """
        Evaluates the strategic value of a non-terminal board state.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The agent's mark.
            config: Game configuration for window scanning.
            
        Returns:
            float: A weighted score representing board 'goodness'.
        """
        score = 0
        enemy = 1 if piece == 2 else 2
        
        def evaluate_window(window, p, e):
            w_score = 0
            # Prioritize wins
            if window.count(p) == 4: w_score += 1000000
            # Reward setups
            elif window.count(p) == 3 and window.count(0) == 1: w_score += 100
            elif window.count(p) == 2 and window.count(0) == 2: w_score += 10
            # Penalize enemy threats heavily
            if window.count(e) == 3 and window.count(0) == 1: w_score -= 10000
            if window.count(e) == 4: w_score -= 1000000
            return w_score

        # Scan Horizontal
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window(list(grid[r, c:c+config.inarow]), piece, enemy)
        # Scan Vertical
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow-1)):
                score += evaluate_window(list(grid[r:r+config.inarow, c]), piece, enemy)
        # Scan Diagonals
        for r in range(config.rows - (config.inarow-1)):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r+i, c+i] for i in range(config.inarow)], piece, enemy)
        for r in range(config.inarow-1, config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r-i, c+i] for i in range(config.inarow)], piece, enemy)
        return score

    # --- CORE: ALPHA-BETA RECURSION ---
    def alphabeta(grid, depth, alpha, beta, is_maximizing, piece, config):
        """
        Executes the recursive search with pruning logic.
        
        Args:
            grid (np.ndarray): Current hypothetical board.
            depth (int): Current depth in search tree.
            alpha (float): Best score found for Maximizer.
            beta (float): Best score found for Minimizer.
            is_maximizing (bool): Whose turn it is in the simulation.
            piece (int): The agent's mark.
            config: Game configuration.
            
        Returns:
            float: The heuristic or terminal value of the branch.
        """
        enemy = 1 if piece == 2 else 2
        
        # Base Cases
        if local_is_win(grid, piece, config): return 1000000
        if local_is_win(grid, enemy, config): return -1000000
        if depth == 0: return local_get_score(grid, piece, config)
        
        valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        if not valid_moves: return 0

        if is_maximizing:
            value = -float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, piece, config)
                value = max(value, alphabeta(temp_grid, depth - 1, alpha, beta, False, piece, config))
                alpha = max(alpha, value)
                if alpha >= beta: break 
            return value
        else:
            value = float('inf')
            for col in valid_moves:
                temp_grid = local_drop_piece(grid, col, enemy, config)
                value = min(value, alphabeta(temp_grid, depth - 1, alpha, beta, True, piece, config))
                beta = min(beta, value)
                if alpha >= beta: break
            return value

    # --- AGENT EXECUTION ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark
    
    # 1. Get and sort moves (Center-out heuristic)
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    valid_moves = sorted(valid_moves, key=lambda x: abs(x - (configuration.columns // 2)))

    best_score = -float('inf')
    best_move = valid_moves[0]
    
    # 2. Root-Level Pruning: Initialize Alpha and Beta at the top
    alpha = -float('inf')
    beta = float('inf')
    
    for col in valid_moves:
        temp_grid = local_drop_piece(grid, col, me, configuration)
        # Check the score of this move
        score = alphabeta(temp_grid, DEPTH-1, alpha, beta, False, me, configuration)
        
        if score > best_score:
            best_score = score
            best_move = col
        
        # Update alpha for the next column check
        alpha = max(alpha, best_score)
            
    return int(best_move)

## AlphaBeta (Depth=6) + sort ordering inside alphabeta

In [5]:
def alphabeta_d6_agent(observation, configuration):
    """
    A Level 4 Strategic AI using Depth-Limited Minimax with Alpha-Beta Pruning.
    
    This agent evaluates the game tree to a specified depth, using Alpha-Beta
    pruning to discard branches that cannot improve the final decision. It 
    prioritizes central columns to maximize pruning efficiency.
    
    Args:
        observation: Kaggle object containing 'board' (1D list) and 'mark' (1 or 2).
        configuration: Kaggle object containing 'rows', 'columns', and 'inarow'.
        
    Returns:
        int: The selected column index (0 to configuration.columns-1).
    """
    import numpy as np

    # --- SETTINGS ---
    # Alpha-beta typically allows for Depth 4-5 within the 2s time limit.
    DEPTH = 6 

    # --- HELPER: DROP PIECE ---
    def local_drop_piece(grid, col, piece, config):
        """
        Simulates the gravity-fed 'drop' of a piece into the board.
        
        Args:
            grid (np.ndarray): The current 2D board state.
            col (int): The column index to drop the piece into.
            piece (int): The player's mark (1 or 2).
            config: Game configuration for board dimensions.
            
        Returns:
            np.ndarray: A copy of the grid with the new piece placed.
        """
        next_grid = grid.copy()
        for r in range(config.rows - 1, -1, -1):
            if next_grid[r][col] == 0:
                next_grid[r][col] = piece
                return next_grid
        return next_grid

    # --- HELPER: IS WIN ---
    def local_is_win(grid, piece, config):
        """
        Determines if the specified piece has achieved the required N-in-a-row.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The player mark to check (1 or 2).
            config: Game configuration for 'inarow' requirement.
            
        Returns:
            bool: True if a winning connection is found, False otherwise.
        """
        # Horizontal check
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r, c+i] == piece for i in range(config.inarow)): return True
        # Vertical check
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow - 1)):
                if all(grid[r+i, c] == piece for i in range(config.inarow)): return True
        # Positive Diagonal check
        for r in range(config.rows - (config.inarow - 1)):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r+i, c+i] == piece for i in range(config.inarow)): return True
        # Negative Diagonal check
        for r in range(config.inarow - 1, config.rows):
            for c in range(config.columns - (config.inarow - 1)):
                if all(grid[r-i, c+i] == piece for i in range(config.inarow)): return True
        return False

    # --- HELPER: HEURISTIC SCORER ---
    def local_get_score(grid, piece, config):
        """
        Evaluates the strategic value of a non-terminal board state.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The agent's mark.
            config: Game configuration for window scanning.
            
        Returns:
            float: A weighted score representing board 'goodness'.
        """
        score = 0
        enemy = 1 if piece == 2 else 2
        
        def evaluate_window(window, p, e):
            w_score = 0
            # Prioritize wins
            if window.count(p) == 4: w_score += 1000000
            # Reward setups
            elif window.count(p) == 3 and window.count(0) == 1: w_score += 100
            elif window.count(p) == 2 and window.count(0) == 2: w_score += 10
            # Penalize enemy threats heavily
            if window.count(e) == 3 and window.count(0) == 1: w_score -= 10000
            if window.count(e) == 4: w_score -= 1000000
            return w_score

        # Scan Horizontal
        for r in range(config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window(list(grid[r, c:c+config.inarow]), piece, enemy)
        # Scan Vertical
        for c in range(config.columns):
            for r in range(config.rows - (config.inarow-1)):
                score += evaluate_window(list(grid[r:r+config.inarow, c]), piece, enemy)
        # Scan Diagonals
        for r in range(config.rows - (config.inarow-1)):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r+i, c+i] for i in range(config.inarow)], piece, enemy)
        for r in range(config.inarow-1, config.rows):
            for c in range(config.columns - (config.inarow-1)):
                score += evaluate_window([grid[r-i, c+i] for i in range(config.inarow)], piece, enemy)
        return score

    # --- CORE: ALPHA-BETA RECURSION ---
    def alphabeta(grid, depth, alpha, beta, is_maximizing, piece, config):
        """
        Executes the recursive search with pruning logic.
        
        Args:
            grid (np.ndarray): Current hypothetical board.
            depth (int): Current depth in search tree.
            alpha (float): Best score found for Maximizer.
            beta (float): Best score found for Minimizer.
            is_maximizing (bool): Whose turn it is in the simulation.
            piece (int): The agent's mark.
            config: Game configuration.
            
        Returns:
            float: The heuristic or terminal value of the branch.
        """
        enemy = 1 if piece == 2 else 2
        
        # Base Cases
        if local_is_win(grid, piece, config): return 1000000
        if local_is_win(grid, enemy, config): return -1000000
        if depth == 0: return local_get_score(grid, piece, config)
        
        valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        if not valid_moves: return 0
        # Use the "Static" Center-Out Sort. 
        valid_moves = sorted(valid_moves, key=lambda x: abs(x - (config.columns // 2)))

        if is_maximizing:
            value = -float('inf')
            for i, col in enumerate(valid_moves):
                temp_grid = local_drop_piece(grid, col, piece, config)
                value = max(value, alphabeta(temp_grid, depth - 1, alpha, beta, False, piece, config))
                alpha = max(alpha, value)
                if alpha >= beta: 
                    # Increment pruning count for remaining moves in this branch
                    break 
            return value
        else:
            value = float('inf')
            for i, col in enumerate(valid_moves):
                temp_grid = local_drop_piece(grid, col, enemy, config)
                value = min(value, alphabeta(temp_grid, depth - 1, alpha, beta, True, piece, config))
                beta = min(beta, value)
                if alpha >= beta: 
                    break
            return value

    # --- AGENT EXECUTION ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark

    # 1. Get and sort moves (Center-out heuristic)
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    valid_moves = sorted(valid_moves, key=lambda x: abs(x - (configuration.columns // 2)))

    best_score = -float('inf')
    best_move = valid_moves[0]
    
    # 2. Root-Level Pruning: Initialize Alpha and Beta at the top
    alpha = -float('inf')
    beta = float('inf')
    
    for col in valid_moves:
        temp_grid = local_drop_piece(grid, col, me, configuration)
        # Check the score of this move
        score = alphabeta(temp_grid, DEPTH-1, alpha, beta, False, me, configuration)
        
        if score > best_score:
            best_score = score
            best_move = col
        
        # Update alpha for the next column check
        alpha = max(alpha, best_score)
    
    return int(best_move)

## AlphaBeta (D=6, Optimized Speed) 
after speed optimizations, also added depth weighting in score to favor faster path to win

In [14]:
def alphabeta_d6_speed_agent(observation, configuration):
    """
    A Level 4 Strategic AI using Depth-Limited Minimax with Alpha-Beta Pruning.
    
    This agent evaluates the game tree to a specified depth, using Alpha-Beta
    pruning to discard branches that cannot improve the final decision. It 
    prioritizes central columns to maximize pruning efficiency.
    
    Args:
        observation: Kaggle object containing 'board' (1D list) and 'mark' (1 or 2).
        configuration: Kaggle object containing 'rows', 'columns', and 'inarow'.
        
    Returns:
        int: The selected column index (0 to configuration.columns-1).
    """
    import numpy as np
    import time

    # --- SETTINGS ---
    start_time = time.time()
    # We use a dictionary for counters so inner functions can modify them
    # stats = {"nodes_evaluated": 0, "nodes_pruned": 0}
    DEPTH = 6 

    # --- HELPER: IS WIN ---
    def is_win_local(grid, row, col, piece, config):
        # 1. Cache configuration values
        n = config.inarow
        rows = config.rows
        cols = config.columns
        target = (piece,) * n
        
        # 2. VERTICAL (Check only downwards)
        # We only need to check down because pieces are placed from bottom-up
        if row <= rows - n:
            if tuple(grid[row:row+n, col]) == target:
                return True
    
        # 3. HORIZONTAL
        # Range is (n-1) steps left and (n-1) steps right
        c_start, c_end = max(0, col - (n - 1)), min(cols, col + n)
        row_segment = tuple(grid[row, c_start:c_end])
        if len(row_segment) >= n:
            for i in range(len(row_segment) - (n - 1)):
                if row_segment[i:i+n] == target: return True
    
        # 4. POSITIVE DIAGONAL (/)
        pos_diag = []
        for i in range(-(n - 1), n):
            r, c = row - i, col + i
            if 0 <= r < rows and 0 <= c < cols:
                pos_diag.append(grid[r, c])
        if len(pos_diag) >= n:
            pos_diag = tuple(pos_diag)
            for i in range(len(pos_diag) - (n - 1)):
                if pos_diag[i:i+n] == target: return True
    
        # 5. NEGATIVE DIAGONAL (\)
        neg_diag = []
        for i in range(-(n - 1), n):
            r, c = row + i, col + i
            if 0 <= r < rows and 0 <= c < cols:
                neg_diag.append(grid[r, c])
        if len(neg_diag) >= n:
            neg_diag = tuple(neg_diag)
            for i in range(len(neg_diag) - (n - 1)):
                if neg_diag[i:i+n] == target: return True
    
        return False

    # --- HELPER: HEURISTIC SCORER ---
    def local_get_score(grid, piece, config):
        """
        Evaluates the strategic value of a non-terminal board state.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The agent's mark.
            config: Game configuration for window scanning.
            
        Returns:
            float: A weighted score representing board 'goodness'.
        """
        score = 0
        enemy = 1 if piece == 2 else 2

        # cache for faster access
        rows = config.rows
        cols = config.columns
        n = config.inarow
        
        def evaluate_window(window, p, e):
            w_score = 0
            # Prioritize wins
            if window.count(p) == 4: w_score += 1000000
            # Reward setups
            elif window.count(p) == 3 and window.count(0) == 1: w_score += 100
            elif window.count(p) == 2 and window.count(0) == 2: w_score += 10
            # Penalize enemy threats heavily
            if window.count(e) == 3 and window.count(0) == 1: w_score -= 10000
            if window.count(e) == 4: w_score -= 1000000
            return w_score

        # Scan Horizontal
        for r in range(rows):
            row_list = list(grid[r, :]) # Convert ONLY ONCE per row
            for c in range(cols - (n-1)):
                # Slicing a list is much faster than slicing an array + list() conversion
                score += evaluate_window(row_list[c:c+n], piece, enemy)
        # Scan Vertical
        for c in range(cols):
            col_list = list(grid[:, c]) # Convert ONLY ONCE per column
            for r in range(rows - (n-1)):
                score += evaluate_window(col_list[r:r+n], piece, enemy)
        # Scan Diagonals
        for r in range(rows - (n-1)):
            for c in range(cols - (n-1)):
                score += evaluate_window([grid[r+i, c+i] for i in range(n)], piece, enemy)
        for r in range(n-1, rows):
            for c in range(cols - (n-1)):
                score += evaluate_window([grid[r-i, c+i] for i in range(n)], piece, enemy)
        return score

    # --- CORE: ALPHA-BETA RECURSION ---
    def alphabeta(grid, depth, alpha, beta, is_maximizing, piece, config, last_row=None, last_col=None):
        """
        Executes the recursive search with pruning logic.
        
        Args:
            grid (np.ndarray): Current hypothetical board.
            depth (int): Current depth in search tree.
            alpha (float): Best score found for Maximizer.
            beta (float): Best score found for Minimizer.
            is_maximizing (bool): Whose turn it is in the simulation.
            piece (int): The agent's mark.
            config: Game configuration.
            last_row: the row where the last piece landed.
            last_col: the col where the last piece landed.
            
        Returns:
            float: The heuristic or terminal value of the branch.
        """
        # stats["nodes_evaluated"] += 1
        enemy = 1 if piece == 2 else 2
        
        # Check if the last move resulted in a win
        if last_row is not None:
            # Who made the last move? 
            # If it's currently maximizing turn, the previous (minimizing) move was the enemy.
            prev_piece = enemy if is_maximizing else piece
            if is_win_local(grid, last_row, last_col, prev_piece, config):
                # Weighting by depth ensures we pick the FASTEST win
                return (1000000 + depth) if prev_piece == piece else (-1000000 - depth)
        # Reached the end node of the tree search, return heuristics score
        if depth == 0: return local_get_score(grid, piece, config)
        
        valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        if not valid_moves: return 0
        # Use the "Static" Center-Out Sort to maximize alpha-beta pruning
        valid_moves = sorted(valid_moves, key=lambda x: abs(x - (config.columns // 2)))

        if is_maximizing:
            value = -float('inf')
            for i, col in enumerate(valid_moves):
                # Find the row for this drop
                row = next(r for r in range(config.rows-1, -1, -1) if grid[r][col] == 0)
                grid[row][col] = piece # Drop
                
                value = max(value, alphabeta(grid, depth - 1, alpha, beta, False, piece, config, row, col))
                grid[row][col] = 0     # Undo the drop
                
                alpha = max(alpha, value)
                if alpha >= beta: 
                    # Increment pruning count for remaining moves in this branch
                    #stats["nodes_pruned"] += (len(valid_moves) - (i + 1))
                    break 
            return value
        else:
            value = float('inf')
            for i, col in enumerate(valid_moves):
                # Find the row for this drop
                row = next(r for r in range(config.rows-1, -1, -1) if grid[r][col] == 0)
                grid[row][col] = enemy # Drop
                
                value = min(value, alphabeta(grid, depth - 1, alpha, beta, True, piece, config, row, col))
                grid[row][col] = 0     # Undo the drop
                
                beta = min(beta, value)
                if alpha >= beta: 
                    #stats["nodes_pruned"] += (len(valid_moves) - (i + 1))
                    break
            return value

    # --- AGENT EXECUTION ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark

    # 1. Get and sort moves (Center-out heuristic)
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    valid_moves = sorted(valid_moves, key=lambda x: abs(x - (configuration.columns // 2)))

    # # 1. Killer Sort the moves to prioritize better ones first to optimize alpha-beta pruning
    # # 1a. Get valid moves
    # valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    # # 1b. Define the Priority Function (Killer Heuristic)
    # def move_priority(col):
    #     enemy = 1 if me == 2 else 2
    #     # Priority 1: Winning move (Highest)
    #     if local_is_win(local_drop_piece(grid, col, me, configuration), me, configuration):
    #         return 100
    #     # Priority 2: Blocking move (Critical)
    #     if local_is_win(local_drop_piece(grid, col, enemy, configuration), enemy, configuration):
    #         return 50
    #     # Priority 3: Center-out (Standard strategy)
    #     return -abs(col - (configuration.columns // 2))
    # # 1c. Sort moves by priority (Highest priority first)
    # valid_moves = sorted(valid_moves, key=move_priority, reverse=True)

    best_score = -float('inf')
    best_move = valid_moves[0]
    
    # 2. Root-Level Pruning: Initialize Alpha and Beta at the top
    alpha = -float('inf')
    beta = float('inf')
    
    for col in valid_moves:
        # Find the row manually of where the piece dropped to
        row = next(r for r in range(configuration.rows-1, -1, -1) if grid[r][col] == 0)
        grid[row][col] = me
        # Check the score of this move
        score = alphabeta(grid, DEPTH-1, alpha, beta, False, me, configuration, row, col)
        grid[row][col] = 0 # Undo
        
        if score > best_score:
            best_score = score
            best_move = col
        
        # Update alpha for the next column check
        alpha = max(alpha, best_score)

    # --- REPORTING ---
    # elapsed = time.time() - start_time
    # total_possible = stats["nodes_evaluated"] + stats["nodes_pruned"]
    # prune_rate = (stats["nodes_pruned"] / total_possible) * 100 if total_possible > 0 else 0
    # nps = stats["nodes_evaluated"] / elapsed if elapsed > 0 else 0
    # print(f"Turn Time: {elapsed:.3f}s | NPS: {nps:.0f} | Nodes Evaluated: {stats['nodes_evaluated']} | "
    #       f"Pruned: {stats['nodes_pruned']} ({prune_rate:.1f}%)")
    
    return int(best_move)

## Alpha Beta (Experimental)

In [9]:
def alphabeta_instrumented_agent(observation, configuration):
    """
    A Level 4 Strategic AI using Depth-Limited Minimax with Alpha-Beta Pruning.
    
    This agent evaluates the game tree to a specified depth, using Alpha-Beta
    pruning to discard branches that cannot improve the final decision. It 
    prioritizes central columns to maximize pruning efficiency.
    
    Args:
        observation: Kaggle object containing 'board' (1D list) and 'mark' (1 or 2).
        configuration: Kaggle object containing 'rows', 'columns', and 'inarow'.
        
    Returns:
        int: The selected column index (0 to configuration.columns-1).
    """
    import numpy as np
    import time

    # --- SETTINGS ---
    start_time = time.time()
    # We use a dictionary for counters so inner functions can modify them
    stats = {"nodes_evaluated": 0, "nodes_pruned": 0}
    DEPTH = 6 

    # --- HELPER: IS WIN ---
    def is_win_local(grid, row, col, piece, config):
        # 1. Cache configuration values
        n = config.inarow
        rows = config.rows
        cols = config.columns
        target = (piece,) * n
        
        # 2. VERTICAL (Check only downwards)
        # We only need to check down because pieces are placed from bottom-up
        if row <= rows - n:
            if tuple(grid[row:row+n, col]) == target:
                return True
    
        # 3. HORIZONTAL
        # Range is (n-1) steps left and (n-1) steps right
        c_start, c_end = max(0, col - (n - 1)), min(cols, col + n)
        row_segment = tuple(grid[row, c_start:c_end])
        if len(row_segment) >= n:
            for i in range(len(row_segment) - (n - 1)):
                if row_segment[i:i+n] == target: return True
    
        # 4. POSITIVE DIAGONAL (/)
        pos_diag = []
        for i in range(-(n - 1), n):
            r, c = row - i, col + i
            if 0 <= r < rows and 0 <= c < cols:
                pos_diag.append(grid[r, c])
        if len(pos_diag) >= n:
            pos_diag = tuple(pos_diag)
            for i in range(len(pos_diag) - (n - 1)):
                if pos_diag[i:i+n] == target: return True
    
        # 5. NEGATIVE DIAGONAL (\)
        neg_diag = []
        for i in range(-(n - 1), n):
            r, c = row + i, col + i
            if 0 <= r < rows and 0 <= c < cols:
                neg_diag.append(grid[r, c])
        if len(neg_diag) >= n:
            neg_diag = tuple(neg_diag)
            for i in range(len(neg_diag) - (n - 1)):
                if neg_diag[i:i+n] == target: return True
    
        return False

    # --- HELPER: HEURISTIC SCORER ---
    def local_get_score(grid, piece, config):
        """
        Evaluates the strategic value of a non-terminal board state.
        
        Args:
            grid (np.ndarray): The 2D board state.
            piece (int): The agent's mark.
            config: Game configuration for window scanning.
            
        Returns:
            float: A weighted score representing board 'goodness'.
        """
        score = 0
        enemy = 1 if piece == 2 else 2

        # cache for faster access
        rows = config.rows
        cols = config.columns
        n = config.inarow
        
        def evaluate_window(window, p, e):
            w_score = 0
            # Prioritize wins
            if window.count(p) == 4: w_score += 1000000
            # Reward setups
            elif window.count(p) == 3 and window.count(0) == 1: w_score += 100
            elif window.count(p) == 2 and window.count(0) == 2: w_score += 10
            # Penalize enemy threats heavily
            if window.count(e) == 3 and window.count(0) == 1: w_score -= 10000
            if window.count(e) == 4: w_score -= 1000000
            return w_score

        # Scan Horizontal
        for r in range(rows):
            row_list = list(grid[r, :]) # Convert ONLY ONCE per row
            for c in range(cols - (n-1)):
                # Slicing a list is much faster than slicing an array + list() conversion
                score += evaluate_window(row_list[c:c+n], piece, enemy)
        # Scan Vertical
        for c in range(cols):
            col_list = list(grid[:, c]) # Convert ONLY ONCE per column
            for r in range(rows - (n-1)):
                score += evaluate_window(col_list[r:r+n], piece, enemy)
        # Scan Diagonals
        for r in range(rows - (n-1)):
            for c in range(cols - (n-1)):
                score += evaluate_window([grid[r+i, c+i] for i in range(n)], piece, enemy)
        for r in range(n-1, rows):
            for c in range(cols - (n-1)):
                score += evaluate_window([grid[r-i, c+i] for i in range(n)], piece, enemy)
        return score

    # --- CORE: ALPHA-BETA RECURSION ---
    def alphabeta(grid, depth, alpha, beta, is_maximizing, piece, config, last_row=None, last_col=None):
        """
        Executes the recursive search with pruning logic.
        
        Args:
            grid (np.ndarray): Current hypothetical board.
            depth (int): Current depth in search tree.
            alpha (float): Best score found for Maximizer.
            beta (float): Best score found for Minimizer.
            is_maximizing (bool): Whose turn it is in the simulation.
            piece (int): The agent's mark.
            config: Game configuration.
            last_row: the row where the last piece landed.
            last_col: the col where the last piece landed.
            
        Returns:
            float: The heuristic or terminal value of the branch.
        """
        stats["nodes_evaluated"] += 1
        enemy = 1 if piece == 2 else 2
        
        # Check if the last move resulted in a win
        if last_row is not None:
            # Who made the last move? 
            # If it's currently maximizing turn, the previous (minimizing) move was the enemy.
            prev_piece = enemy if is_maximizing else piece
            if is_win_local(grid, last_row, last_col, prev_piece, config):
                # Weighting by depth ensures we pick the FASTEST win
                return (1000000 + depth) if prev_piece == piece else (-1000000 - depth)
        # Reached the end node of the tree search, return heuristics score
        if depth == 0: return local_get_score(grid, piece, config)
        
        valid_moves = [c for c in range(config.columns) if grid[0][c] == 0]
        if not valid_moves: return 0
        # Use the "Static" Center-Out Sort to maximize alpha-beta pruning
        valid_moves = sorted(valid_moves, key=lambda x: abs(x - (config.columns // 2)))

        if is_maximizing:
            value = -float('inf')
            for i, col in enumerate(valid_moves):
                # Find the row for this drop
                row = next(r for r in range(config.rows-1, -1, -1) if grid[r][col] == 0)
                grid[row][col] = piece # Drop
                
                value = max(value, alphabeta(grid, depth - 1, alpha, beta, False, piece, config, row, col))
                grid[row][col] = 0     # Undo the drop
                
                alpha = max(alpha, value)
                if alpha >= beta: 
                    # Increment pruning count for remaining moves in this branch
                    stats["nodes_pruned"] += (len(valid_moves) - (i + 1))
                    break 
            return value
        else:
            value = float('inf')
            for i, col in enumerate(valid_moves):
                # Find the row for this drop
                row = next(r for r in range(config.rows-1, -1, -1) if grid[r][col] == 0)
                grid[row][col] = enemy # Drop
                
                value = min(value, alphabeta(grid, depth - 1, alpha, beta, True, piece, config, row, col))
                grid[row][col] = 0     # Undo the drop
                
                beta = min(beta, value)
                if alpha >= beta: 
                    stats["nodes_pruned"] += (len(valid_moves) - (i + 1))
                    break
            return value

    # --- AGENT EXECUTION ---
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    me = observation.mark

    # 1. Get and sort moves (Center-out heuristic)
    valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    valid_moves = sorted(valid_moves, key=lambda x: abs(x - (configuration.columns // 2)))

    # # 1. Killer Sort the moves to prioritize better ones first to optimize alpha-beta pruning
    # # 1a. Get valid moves
    # valid_moves = [c for c in range(configuration.columns) if observation.board[c] == 0]
    # # 1b. Define the Priority Function (Killer Heuristic)
    # def move_priority(col):
    #     enemy = 1 if me == 2 else 2
    #     # Priority 1: Winning move (Highest)
    #     if local_is_win(local_drop_piece(grid, col, me, configuration), me, configuration):
    #         return 100
    #     # Priority 2: Blocking move (Critical)
    #     if local_is_win(local_drop_piece(grid, col, enemy, configuration), enemy, configuration):
    #         return 50
    #     # Priority 3: Center-out (Standard strategy)
    #     return -abs(col - (configuration.columns // 2))
    # # 1c. Sort moves by priority (Highest priority first)
    # valid_moves = sorted(valid_moves, key=move_priority, reverse=True)

    best_score = -float('inf')
    best_move = valid_moves[0]
    
    # 2. Root-Level Pruning: Initialize Alpha and Beta at the top
    alpha = -float('inf')
    beta = float('inf')
    
    for col in valid_moves:
        # Find the row manually of where the piece dropped to
        row = next(r for r in range(configuration.rows-1, -1, -1) if grid[r][col] == 0)
        grid[row][col] = me
        # Check the score of this move
        score = alphabeta(grid, DEPTH-1, alpha, beta, False, me, configuration, row, col)
        grid[row][col] = 0 # Undo
        
        if score > best_score:
            best_score = score
            best_move = col
        
        # Update alpha for the next column check
        alpha = max(alpha, best_score)

    # --- REPORTING ---
    elapsed = time.time() - start_time
    total_possible = stats["nodes_evaluated"] + stats["nodes_pruned"]
    prune_rate = (stats["nodes_pruned"] / total_possible) * 100 if total_possible > 0 else 0
    nps = stats["nodes_evaluated"] / elapsed if elapsed > 0 else 0
    print(f"Turn Time: {elapsed:.3f}s | NPS: {nps:.0f} | Nodes Evaluated: {stats['nodes_evaluated']} | "
          f"Pruned: {stats['nodes_pruned']} ({prune_rate:.1f}%)")
    
    return int(best_move)

# Arena Play
Play a game between two agents

In [13]:
from kaggle_environments import make
env = make("connectx", debug=True)
# env.run([alphabeta_instrumented_agent, alphabeta_d4_agent])
env.run([alphabeta_d7_agent, alphabeta_d4_agent])
env.render(mode="ipython", width=500, height=450)

Error: name 'stats' is not defined


Play Your Agent

In [ ]:
# "None" represents which agent you'll manually play as (first or second player).
from kaggle_environments import make
env = make("connectx", debug=True)
env.play([None, "negamax"], width=500, height=450)

# Evaluation
Evaluate the agent in batch test runs against benchmark agents

In [11]:
def agent_battle(agent1, agent2, n_rounds=100):
    # Get names for printing
    name1 = agent1.__name__ if hasattr(agent1, '__name__') else str(agent1)
    name2 = agent2.__name__ if hasattr(agent2, '__name__') else str(agent2)
    
    config = {'rows': 6, 'columns': 7, 'inarow': 4}

    # Run the rounds
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds // 2)
    outcomes += [[b, a] for [a, b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds - n_rounds // 2)]
    # Calculate stats based on corrected rewards
    # outcomes.count([1, 0]) = Agent 1 wins
    # outcomes.count([0.5, 0.5]) = Ties
    # outcomes.count([None, 1]) = Agent 1 crashed
    
    a1_wins = outcomes.count([1, 0])
    a2_wins = outcomes.count([0, 1])
    ties = outcomes.count([0.5, 0.5])
    invalid = len(outcomes) - (a1_wins + a2_wins + ties)
    
    print(f"--- Results: {name1} vs {name2} ---")
    print(f"{name1} Win Rate: {np.round(a1_wins / len(outcomes), 2)}")
    print(f"{name2} Win Rate: {np.round(a2_wins / len(outcomes), 2)}")
    print(f"Ties: {np.round(ties / len(outcomes), 2)}")
    print(f"Invalid Plays (Crashes): {np.round(invalid / len(outcomes), 2)}")

## Random vs Negamax

In [ ]:
n_rounds = 10
print("Random vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(random_agent, "negamax", n_rounds)

## Leftmost vs Random

In [ ]:
n_rounds = 100
print("Random vs Leftmost")
print(str(n_rounds) + " rounds...")
agent_battle(random_agent, leftmost_agent, n_rounds)

## Negamax vs Negamax

In [ ]:
n_rounds = 100
print("Negamax vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle("negamax", "negamax", n_rounds)

## Reactive vs Random

In [ ]:
n_rounds = 100
print("Reactive vs Random")
print(str(n_rounds) + " rounds...")
agent_battle(reactive_agent, random_agent, n_rounds)

## Reactive vs Leftmost

In [ ]:
n_rounds = 100
print("Reactive vs Leftmost")
print(str(n_rounds) + " rounds...")
agent_battle(reactive_agent, leftmost_agent, n_rounds)

## Reactive vs Negamax

In [ ]:
n_rounds = 100
print("Reactive vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(reactive_agent, "negamax", n_rounds)

## Minimax

In [ ]:
n_rounds = 100
print("MiniMax vs Random")
print(str(n_rounds) + " rounds...")
agent_battle(minimax_agent, random_agent, n_rounds)

In [ ]:
n_rounds = 100
print("MiniMax vs Reactive")
print(str(n_rounds) + " rounds...")
agent_battle(minimax_agent, reactive_agent, n_rounds)

In [12]:
n_rounds = 100
print("MiniMax vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(minimax_agent, "negamax", n_rounds)

MiniMax vs Negamax
100 rounds...
--- Results: minimax_agent vs negamax ---
minimax_agent Win Rate: 0.5
negamax Win Rate: 0.5
Ties: 0.0
Invalid Plays (Crashes): 0.0


### Alpha-Beta

In [7]:
n_rounds = 100
print("AlphaBeta vs Reactive")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_agent, reactive_agent, n_rounds)

AlphaBeta vs Reactive
100 rounds...
--- Results: alphabeta_agent vs reactive_agent ---
alphabeta_agent Win Rate: 0.98
reactive_agent Win Rate: 0.01
Ties: 0.01
Invalid Plays (Crashes): 0.0


In [9]:
n_rounds = 2
print("AlphaBeta(d=4) vs Minimax(depth=3)")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_d4_agent, minimax_agent, n_rounds)

AlphaBeta(d=4) vs Minimax(depth=3)
2 rounds...
--- Results: alphabeta_d4_agent vs minimax_agent ---
alphabeta_d4_agent Win Rate: 0.5
minimax_agent Win Rate: 0.5
Ties: 0.0
Invalid Plays (Crashes): 0.0


In [9]:
n_rounds = 2
print("AlphaBeta vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_agent, "negamax", n_rounds)

AlphaBeta vs Negamax
100 rounds...
--- Results: alphabeta_agent vs negamax ---
alphabeta_agent Win Rate: 1.0
negamax Win Rate: 0.0
Ties: 0.0
Invalid Plays (Crashes): 0.0


In [10]:
n_rounds = 2
print("AlphaBeta(D=5) vs Minimax(D=3)")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_d5_agent, minimax_agent, n_rounds)

AlphaBeta(D=5) vs Minimax(D=3)
2 rounds...
--- Results: alphabeta_d5_agent vs minimax_agent ---
alphabeta_d5_agent Win Rate: 0.5
minimax_agent Win Rate: 0.0
Ties: 0.5
Invalid Plays (Crashes): 0.0


In [8]:
n_rounds = 2
print("AlphaBeta(D=5) vs AlphaBeta(D=4)")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_d5_agent, alphabeta_d4_agent, n_rounds)

AlphaBeta(D=5) vs AlphaBeta(D=4)
20 rounds...
--- Results: alphabeta_d5_agent vs alphabeta_d4_agent ---
alphabeta_d5_agent Win Rate: 0.0
alphabeta_d4_agent Win Rate: 0.5
Ties: 0.5
Invalid Plays (Crashes): 0.0


In [12]:
n_rounds = 2
print("AlphaBeta (Instrumented) vs Negamax")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_instrumented_agent, "negamax", n_rounds)

AlphaBeta (Instrumented) vs Negamax
2 rounds...
--- Results: alphabeta_instrumented_agent vs negamax ---
alphabeta_instrumented_agent Win Rate: 0.0
negamax Win Rate: 0.0
Ties: 0.0
Invalid Plays (Crashes): 1.0


In [45]:
n_rounds = 2
print("AlphaBeta(D=6) vs AlphaBeta(D=4)")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_d6_agent, alphabeta_d4_agent, n_rounds)

AlphaBeta(D=6) vs AlphaBeta(D=4)
2 rounds...
--- Results: alphabeta_d6_agent vs alphabeta_d4_agent ---
alphabeta_d6_agent Win Rate: 0.0
alphabeta_d4_agent Win Rate: 0.0
Ties: 0.0
Invalid Plays (Crashes): 1.0


In [15]:
n_rounds = 2
print("AlphaBeta(D=6, optimized) vs AlphaBeta(D=4)")
print(str(n_rounds) + " rounds...")
agent_battle(alphabeta_d6_speed_agent, alphabeta_d4_agent, n_rounds)

AlphaBeta(D=7) vs AlphaBeta(D=4)
2 rounds...
--- Results: alphabeta_d7_agent vs alphabeta_d4_agent ---
alphabeta_d7_agent Win Rate: 0.5
alphabeta_d4_agent Win Rate: 0.0
Ties: 0.5
Invalid Plays (Crashes): 0.0


# Custom 2D Board Rendering

In [ ]:
import numpy as np

def render_board(observation, configuration):
    # Convert the flat list to a 2D numpy array (Matrix)
    # This is exactly how your AI will 'see' the board
    grid = np.array(observation.board).reshape(configuration.rows, configuration.columns)
    
    print("\n  0 1 2 3 4 5 6") # Column headers
    print(" ---------------")
    for row in grid:
        # Replace numbers with clearer symbols: . (empty), X (P1), O (P2)
        row_str = " ".join(['.' if x == 0 else ('X' if x == 1 else 'O') for x in row])
        print(f"| {row_str} |")
    print(" ---------------")

# Test it
from kaggle_environments import make
env = make("connectx", debug=True)
env.run(["random", "random"])

render_board(env.state[0].observation, env.configuration)

# Test Helper Functions

## Optimize Is Win Function

#### Is Win (Python For Implementation) - Original implementation

In [7]:
def is_win_local(grid, row, col, piece, config):
    """
    Checks if the piece just placed at (row, col) creates a win.
    Only checks the row, column, and diagonals intersecting this point.
    """
    # 1. VERTICAL (Only need to check downwards from the current piece)
    count = 0
    # Range is just the 4 slots including/below the current row
    for r in range(row, min(config.rows, row + 4)):
        if grid[r][col] == piece:
            count += 1
            if count == 4: return True
        else:
            break # Vertical can only be downward; if we hit a gap, we stop.

    # 2. HORIZONTAL
    count = 0
    # Check the 7-wide window centered on the new piece
    for c in range(max(0, col - 3), min(config.columns, col + 4)):
        if grid[row][c] == piece:
            count += 1
            if count == 4: return True
        else:
            count = 0 # THE FIX: Reset if not consecutive

    # 3. POSITIVE DIAGONAL (/)
    count = 0
    for i in range(-3, 4):
        r, c = row - i, col + i
        if 0 <= r < config.rows and 0 <= c < config.columns:
            if grid[r][c] == piece:
                count += 1
                if count == 4: return True
            else:
                count = 0 # Reset
    
    # 4. NEGATIVE DIAGONAL (\)
    count = 0
    for i in range(-3, 4):
        r, c = row + i, col + i
        if 0 <= r < config.rows and 0 <= c < config.columns:
            if grid[r][c] == piece:
                count += 1
                if count == 4: return True
            else:
                count = 0 # Reset
                
    return False

#### Is Win (Tupple Implementation)

In [24]:
def is_win_local(grid, row, col, piece, config):
    # 1. Cache configuration values
    n = config.inarow
    rows = config.rows
    cols = config.columns
    target = (piece,) * n
    
    # 2. VERTICAL (Check only downwards)
    # We only need to check down because pieces are placed from bottom-up
    if row <= rows - n:
        if tuple(grid[row:row+n, col]) == target:
            return True

    # 3. HORIZONTAL
    # Range is (n-1) steps left and (n-1) steps right
    c_start, c_end = max(0, col - (n - 1)), min(cols, col + n)
    row_segment = tuple(grid[row, c_start:c_end])
    if len(row_segment) >= n:
        for i in range(len(row_segment) - (n - 1)):
            if row_segment[i:i+n] == target: return True

    # 4. POSITIVE DIAGONAL (/)
    pos_diag = []
    for i in range(-(n - 1), n):
        r, c = row - i, col + i
        if 0 <= r < rows and 0 <= c < cols:
            pos_diag.append(grid[r, c])
    if len(pos_diag) >= n:
        pos_diag = tuple(pos_diag)
        for i in range(len(pos_diag) - (n - 1)):
            if pos_diag[i:i+n] == target: return True

    # 5. NEGATIVE DIAGONAL (\)
    neg_diag = []
    for i in range(-(n - 1), n):
        r, c = row + i, col + i
        if 0 <= r < rows and 0 <= c < cols:
            neg_diag.append(grid[r, c])
    if len(neg_diag) >= n:
        neg_diag = tuple(neg_diag)
        for i in range(len(neg_diag) - (n - 1)):
            if neg_diag[i:i+n] == target: return True

    return False

In [25]:
import numpy as np

def run_win_check_tests():
    # Mock config
    class Config:
        rows = 6
        columns = 7
        inarow = 4
    config = Config()
    
    tests = []

    # TEST 1: Vertical Win
    grid = np.zeros((6, 7))
    grid[5,0]=1; grid[4,0]=1; grid[3,0]=1; grid[2,0]=1
    tests.append(("Vertical Win", is_win_local(grid, 2, 0, 1, config), True))

    # TEST 2: Vertical Gap (The bug you found!)
    grid = np.zeros((6, 7))
    grid[5,0]=1; grid[4,0]=1; grid[3,0]=0; grid[2,0]=1; grid[1,0]=1
    tests.append(("Vertical Gap", is_win_local(grid, 1, 0, 1, config), False))

    # TEST 3: Horizontal Win (Far Right)
    grid = np.zeros((6, 7))
    grid[5,3]=1; grid[5,4]=1; grid[5,5]=1; grid[5,6]=1
    tests.append(("Horizontal Win Right", is_win_local(grid, 5, 6, 1, config), True))

    # TEST 4: Horizontal Gap (Center)
    grid = np.zeros((6, 7))
    grid[5,1]=1; grid[5,2]=1; grid[5,3]=0; grid[5,4]=1; grid[5,5]=1
    tests.append(("Horizontal Gap", is_win_local(grid, 5, 5, 1, config), False))

    # TEST 5: Positive Diagonal Win
    grid = np.zeros((6, 7))
    grid[5,0]=1; grid[4,1]=1; grid[3,2]=1; grid[2,3]=1
    tests.append(("Pos Diagonal Win", is_win_local(grid, 2, 3, 1, config), True))

    # TEST 6: Negative Diagonal Win
    grid = np.zeros((6, 7))
    grid[2,0]=1; grid[3,1]=1; grid[4,2]=1; grid[5,3]=1
    tests.append(("Neg Diagonal Win", is_win_local(grid, 5, 3, 1, config), True))

    # TEST 7: Positive Diagonal Gap Trap (/)
    # (5,0)=1, (4,1)=1, (3,2)=0, (2,3)=1, (1,4)=1
    grid = np.zeros((6, 7))
    grid[5,0]=1; grid[4,1]=1; grid[3,2]=0; grid[2,3]=1; grid[1,4]=1
    tests.append(("Pos-Diag Gap Trap", is_win_local(grid, 1, 4, 1, config), False))

    # TEST 8. Negative Diagonal Gap Trap (\)
    # (0,0)=1, (1,1)=1, (2,2)=0, (3,3)=1, (4,4)=1
    grid = np.zeros((6, 7))
    grid[0,0]=1; grid[1,1]=1; grid[2,2]=0; grid[3,3]=1; grid[4,4]=1
    tests.append(("Neg-Diag Gap Trap", is_win_local(grid, 4, 4, 1, config), False))

    # Results Printout
    print(f"{'Test Case':<25} | {'Passed':<10}")
    print("-" * 40)
    for name, result, expected in tests:
        passed = "✅" if result == expected else "❌ FAIL"
        print(f"{name:<25} | {passed}")

run_win_check_tests()

Test Case                 | Passed    
----------------------------------------
Vertical Win              | ✅
Vertical Gap              | ✅
Horizontal Win Right      | ✅
Horizontal Gap            | ✅
Pos Diagonal Win          | ✅
Neg Diagonal Win          | ✅
Pos-Diag Gap Trap         | ✅
Neg-Diag Gap Trap         | ✅


In [11]:
import numpy as np
import time

# --- COMPETITOR A: THE COUNTER LOOP ---
def is_win_local_loop(grid, row, col, piece, config):
    # (Simplified Horizontal check for the benchmark)
    count = 0
    for c in range(max(0, col - 3), min(config.columns, col + 4)):
        if grid[row][c] == piece:
            count += 1
            if count == 4: return True
        else:
            count = 0
    return False

# --- COMPETITOR B: THE NATIVE TUPLE SEARCH ---
def is_win_local_tuple(grid, row, col, piece, config):
    target = (piece, piece, piece, piece)
    r_start, r_end = max(0, col - 3), min(config.columns, col + 4)
    row_segment = tuple(grid[row, r_start:r_end])
    if len(row_segment) >= 4:
        for i in range(len(row_segment) - 3):
            if row_segment[i:i+4] == target: return True
    return False

# --- BENCHMARK RUNNER ---
def run_benchmark():
    class Config: rows = 6; columns = 7; inarow = 4
    config = Config()
    grid = np.zeros((6, 7))
    grid[5, 1:5] = 1 # A horizontal win
    
    iterations = 100000
    
    print(f"Running {iterations} iterations...")

    # Test Loop
    start = time.time()
    for _ in range(iterations):
        is_win_local_loop(grid, 5, 4, 1, config)
    loop_time = time.time() - start

    # Test Tuple
    start = time.time()
    for _ in range(iterations):
        is_win_local_tuple(grid, 5, 4, 1, config)
    tuple_time = time.time() - start

    print(f"\nResults:")
    print(f"Loop Method:  {loop_time:.4f} seconds")
    print(f"Tuple Method: {tuple_time:.4f} seconds")
    
    improvement = ((loop_time - tuple_time) / loop_time) * 100
    print(f"\nEfficiency Gain: {improvement:.1f}%")

run_benchmark()

Running 100000 iterations...

Results:
Loop Method:  0.4458 seconds
Tuple Method: 0.3064 seconds

Efficiency Gain: 31.3%
